# Lesson 2: Time Series Forecasting & Pattern Mining

## Learning Objectives
- LO8: Make predictions using ARMA/ARIMA models
- LO9: Recognize frequently occurring patterns (time series motifs)

---

## Setup: Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Try to import stumpy for motif detection (if not installed, we'll provide alternative)
try:
    import stumpy
    STUMPY_AVAILABLE = True
except ImportError:
    STUMPY_AVAILABLE = False
    print("Note: stumpy not installed. Install with: pip install stumpy")

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Stumpy available: {STUMPY_AVAILABLE}")

---
## Opening Activity: "Predict the Future" (15 min)

Let's start by visualizing some sales data and making intuitive predictions.

In [ ]:
# Generate synthetic sales data with trend and seasonality
np.random.seed(42)

# 12 months of historical data
months = pd.date_range('2023-01-01', periods=12, freq='MS')
trend = np.linspace(100, 130, 12)
seasonality = 20 * np.sin(2 * np.pi * np.arange(12) / 12)
noise = np.random.normal(0, 5, 12)
sales = trend + seasonality + noise

# Create DataFrame
df_sales = pd.DataFrame({
    'month': months,
    'sales': sales
})

# Visualize
plt.figure(figsize=(12, 6))
plt.plot(df_sales['month'], df_sales['sales'], marker='o', linewidth=2, markersize=10, color='steelblue')
plt.axvline(x=df_sales['month'].iloc[-1], color='red', linestyle='--', linewidth=2, label='Today')
plt.title('Sales Data - Past 12 Months', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Sales (units)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print("\n🤔 Activity:")
print("1. Draw or imagine the next 3 months of sales")
print("2. What patterns do you see in the historical data?")
print("3. What information are you using to make your prediction?")

### 💬 Discussion with Your Neighbor

**Share your predictions:**
- What patterns did you identify? ___________
- What will sales be in month 13? ___________
- What assumptions did you make? ___________

**Key insight:** You're using patterns from the past to predict the future - that's exactly what ARIMA does!

---
## Part 1: ARMA & ARIMA Intuition (35 min)

### Understanding the Components

### 1. AR (AutoRegressive) Model

**Intuition:** "Today's value depends on yesterday's and the day before"

**Formula:** X_t = c + φ₁X_{t-1} + φ₂X_{t-2} + ... + error

**Example analogy:** Your mood today depends on your mood yesterday and the day before

In [ ]:
# Simulate AR processes with different parameters
np.random.seed(42)
n = 200

def simulate_ar(phi, n=200):
    """Simulate AR(1) process: X_t = phi * X_{t-1} + error"""
    x = np.zeros(n)
    x[0] = np.random.normal()
    for t in range(1, n):
        x[t] = phi * x[t-1] + np.random.normal()
    return x

# Different AR processes
ar_positive = simulate_ar(0.8)  # Strong positive correlation
ar_moderate = simulate_ar(0.3)  # Moderate correlation
ar_negative = simulate_ar(-0.5) # Negative correlation

# Visualize
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

axes[0].plot(ar_positive, linewidth=1)
axes[0].set_title('AR(1) with φ = 0.8 (Strong positive correlation)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Value')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ar_moderate, linewidth=1)
axes[1].set_title('AR(1) with φ = 0.3 (Moderate correlation)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Value')
axes[1].grid(True, alpha=0.3)

axes[2].plot(ar_negative, linewidth=1, color='red')
axes[2].set_title('AR(1) with φ = -0.5 (Negative correlation - oscillating)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Value')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("- φ > 0: Values tend to continue in the same direction (smooth trends)")
print("- φ close to 1: Very persistent, slow decay")
print("- φ < 0: Values oscillate (negative correlation)")
print("- φ close to 0: Like random noise")

### 📝 Quick Poll

Which of these time series would have strong AR behavior?

1. Daily temperature: _____ (Yes/No)
2. Rolling a die: _____ (Yes/No)
3. Stock prices: _____ (Yes/No)
4. Lottery numbers: _____ (Yes/No)

### 2. MA (Moving Average) Model

**Intuition:** "Today's value depends on recent shocks/surprises"

**Formula:** X_t = μ + θ₁ε_{t-1} + θ₂ε_{t-2} + ... + ε_t

**Example:** Your mood depends on recent unexpected events (surprises)

In [ ]:
def simulate_ma(theta, n=200):
    """Simulate MA(1) process: X_t = theta * error_{t-1} + error_t"""
    errors = np.random.normal(0, 1, n)
    x = np.zeros(n)
    x[0] = errors[0]
    for t in range(1, n):
        x[t] = theta * errors[t-1] + errors[t]
    return x

# Different MA processes
ma_positive = simulate_ma(0.8)
ma_negative = simulate_ma(-0.8)

# Visualize comparison of AR vs MA
fig, axes = plt.subplots(2, 2, figsize=(15, 8))

axes[0, 0].plot(ar_positive, linewidth=1, color='blue')
axes[0, 0].set_title('AR(1): φ = 0.8', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Value')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(ma_positive, linewidth=1, color='green')
axes[0, 1].set_title('MA(1): θ = 0.8', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(ar_negative, linewidth=1, color='blue')
axes[1, 0].set_title('AR(1): φ = -0.5', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Value')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(ma_negative, linewidth=1, color='green')
axes[1, 1].set_title('MA(1): θ = -0.8', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Time')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Key Difference:")
print("- AR: Long memory - past values directly influence future")
print("- MA: Short memory - only recent shocks matter")
print("- AR processes look smoother and more persistent")
print("- MA processes look more 'choppy' with less persistence")

### 3. ARIMA: Putting It All Together

**ARIMA(p, d, q)**
- **p**: Number of AR (autoregressive) terms
- **d**: Degree of differencing (for stationarity)
- **q**: Number of MA (moving average) terms

**The "I" (Integrated):** Differencing to make data stationary
- d=0: No differencing (data already stationary)
- d=1: First difference (X_t - X_{t-1})
- d=2: Second difference

In [ ]:
# Demonstrate differencing
np.random.seed(42)
n = 200

# Non-stationary series (with trend)
trend = 0.5 * np.arange(n)
noise = np.random.normal(0, 5, n)
non_stationary = trend + noise

# First difference
first_diff = np.diff(non_stationary)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

axes[0].plot(non_stationary, linewidth=1)
axes[0].set_title('Original Series (Non-Stationary with Trend)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Value')
axes[0].grid(True, alpha=0.3)

axes[1].plot(first_diff, linewidth=1, color='orange')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('After First Differencing (Stationary)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Differenced Value')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Differencing removes the trend and makes data stationary!")

---
### 🧪 Experiment 1: Play with AR Parameters (10 min)

In [ ]:
# YOUR TURN: Experiment with different AR parameters
phi_value = 0.9  # Try: 0.3, 0.5, 0.9, 0.99, -0.5, 1.1 (unstable!)

# Simulate
experiment_ar = simulate_ar(phi_value, n=300)

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(experiment_ar, linewidth=1)
plt.title(f'Your AR(1) Process with φ = {phi_value}', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Statistics for φ = {phi_value}:")
print(f"Mean: {np.mean(experiment_ar):.2f}")
print(f"Std Dev: {np.std(experiment_ar):.2f}")
print(f"Min: {np.min(experiment_ar):.2f}")
print(f"Max: {np.max(experiment_ar):.2f}")

print("\n💭 Reflection Questions:")
print("- What happens when φ > 1? (Try it!)")
print("- What happens with negative φ?")
print("- Which φ value creates the smoothest series?")

---
### 🧪 Experiment 2: Fit ARIMA on Real Data (10 min)

Let's work through the complete ARIMA workflow!

In [ ]:
# Generate realistic energy consumption data
np.random.seed(42)
days = pd.date_range('2023-01-01', periods=365, freq='D')
trend = np.linspace(100, 110, 365)
seasonal = 10 * np.sin(2 * np.pi * np.arange(365) / 7)  # Weekly seasonality
noise = np.random.normal(0, 3, 365)
energy = trend + seasonal + noise

df_energy = pd.DataFrame({
    'date': days,
    'consumption': energy
})

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(df_energy['date'], df_energy['consumption'], linewidth=1)
plt.title('Daily Energy Consumption', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Energy (kWh)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Dataset: {len(df_energy)} days of energy consumption data")

#### Step 1: Check Stationarity (ADF Test)

In [ ]:
def adf_test(series, name=''):
    """Perform Augmented Dickey-Fuller test for stationarity"""
    result = adfuller(series.dropna())
    print(f'\n📊 ADF Test Results for {name}:')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'  {key}: {value:.3f}')
    
    if result[1] < 0.05:
        print("\n✅ Result: Series is STATIONARY (p < 0.05)")
    else:
        print("\n❌ Result: Series is NON-STATIONARY (p >= 0.05) - needs differencing!")
    
    return result[1] < 0.05

# Test original series
is_stationary = adf_test(df_energy['consumption'], 'Original Series')

# If not stationary, try differencing
if not is_stationary:
    df_energy['consumption_diff'] = df_energy['consumption'].diff()
    adf_test(df_energy['consumption_diff'], 'First Differenced Series')

#### Step 2: Determine Parameters with ACF/PACF Plots

In [ ]:
# Plot ACF and PACF
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Use differenced series if original wasn't stationary
series_to_plot = df_energy['consumption_diff'].dropna() if not is_stationary else df_energy['consumption']

plot_acf(series_to_plot, lags=30, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')

plot_pacf(series_to_plot, lags=30, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📚 How to read ACF/PACF:")
print("\nPACF (for determining p):")
print("  - Significant spikes = number of AR terms (p)")
print("  - Look for where PACF cuts off")
print("\nACF (for determining q):")
print("  - Significant spikes = number of MA terms (q)")
print("  - Look for where ACF cuts off")
print("\n💡 Tip: Start with small values (1-3) and compare models!")

#### Step 3: Fit ARIMA Model

In [ ]:
# YOUR TURN: Choose parameters based on ACF/PACF
p = 1  # AR order (from PACF)
d = 1  # Differencing order (1 if we needed differencing, 0 if not)
q = 1  # MA order (from ACF)

print(f"\n🔧 Fitting ARIMA({p}, {d}, {q}) model...")

# Fit model
model = ARIMA(df_energy['consumption'], order=(p, d, q))
fitted_model = model.fit()

# Display summary
print("\n" + "="*50)
print("MODEL SUMMARY")
print("="*50)
print(fitted_model.summary())

# Get fitted values
df_energy['fitted'] = fitted_model.fittedvalues

# Visualize fit
plt.figure(figsize=(15, 6))
plt.plot(df_energy['date'], df_energy['consumption'], label='Actual', linewidth=1, alpha=0.7)
plt.plot(df_energy['date'], df_energy['fitted'], label=f'ARIMA({p},{d},{q}) Fit', linewidth=2)
plt.title('ARIMA Model Fit', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Energy Consumption')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate error metrics
residuals = df_energy['consumption'] - df_energy['fitted']
mae = mean_absolute_error(df_energy['consumption'][d:], df_energy['fitted'][d:])
rmse = np.sqrt(mean_squared_error(df_energy['consumption'][d:], df_energy['fitted'][d:]))

print(f"\n📊 Model Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

#### Step 4: Make Forecasts

In [ ]:
# Forecast next 30 days
forecast_steps = 30
forecast = fitted_model.forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=df_energy['date'].iloc[-1] + pd.Timedelta(days=1), periods=forecast_steps, freq='D')

# Get confidence intervals
forecast_df = fitted_model.get_forecast(steps=forecast_steps)
forecast_ci = forecast_df.conf_int()

# Visualize forecast
plt.figure(figsize=(15, 6))

# Historical data
plt.plot(df_energy['date'], df_energy['consumption'], label='Historical', linewidth=2, color='steelblue')

# Forecast
plt.plot(forecast_index, forecast, label='Forecast', linewidth=2, color='red', linestyle='--')

# Confidence intervals
plt.fill_between(forecast_index, 
                 forecast_ci.iloc[:, 0], 
                 forecast_ci.iloc[:, 1], 
                 alpha=0.3, 
                 color='red',
                 label='95% Confidence Interval')

plt.axvline(x=df_energy['date'].iloc[-1], color='black', linestyle=':', linewidth=2, label='Forecast Start')
plt.title(f'ARIMA({p},{d},{q}) Forecast - Next {forecast_steps} Days', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Energy Consumption')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n🔮 Forecast for next {forecast_steps} days:")
forecast_summary = pd.DataFrame({
    'Date': forecast_index,
    'Forecast': forecast.values,
    'Lower CI': forecast_ci.iloc[:, 0].values,
    'Upper CI': forecast_ci.iloc[:, 1].values
})
print(forecast_summary.head(10))

### 💭 Reflection Questions:

1. Why might your ARIMA parameters differ from your neighbor's? ___________

2. What does the PACF plot tell you about the data? ___________

3. How confident are you in the forecast for day 1 vs day 30? Why? ___________

4. Try different (p,d,q) values - which combination gives the lowest MAE? ___________

---
## ☕ BREAK (10 minutes)
---

---
## Part 2: Time Series Forecasting in Practice (25 min)

### Case-Based Learning

Work in groups of 3. Each group will receive ONE scenario below.

### Scenario Selection

**Choose your scenario:**

1. **Retail**: Predict demand for sunscreen next summer
2. **Energy**: Predict electricity consumption for the coming week
3. **Maintenance**: Predict when machine maintenance is needed
4. **Finance**: Predict revenue for the next quarter

In [ ]:
# Generate different datasets for each scenario
np.random.seed(42)

def create_scenario_data(scenario_type):
    """Create realistic data for different scenarios"""
    
    if scenario_type == 'retail':
        # Sunscreen sales - strong yearly seasonality
        days = pd.date_range('2021-01-01', periods=1095, freq='D')  # 3 years
        trend = np.linspace(100, 150, 1095)
        yearly_season = 80 * np.sin(2 * np.pi * np.arange(1095) / 365 - np.pi/2)  # Peak in summer
        noise = np.random.normal(0, 10, 1095)
        values = trend + yearly_season + noise
        values = np.maximum(values, 5)  # No negative sales
        label = 'Sunscreen Sales (units)'
        
    elif scenario_type == 'energy':
        # Electricity consumption - weekly + daily patterns
        hours = pd.date_range('2023-01-01', periods=24*90, freq='H')  # 90 days hourly
        days = hours
        trend = np.linspace(500, 520, len(hours))
        daily_season = 100 * np.sin(2 * np.pi * np.arange(len(hours)) / 24)
        weekly_season = 50 * np.sin(2 * np.pi * np.arange(len(hours)) / (24*7))
        noise = np.random.normal(0, 20, len(hours))
        values = trend + daily_season + weekly_season + noise
        label = 'Electricity Consumption (kWh)'
        
    elif scenario_type == 'maintenance':
        # Machine vibration - increasing trend before failure
        hours = pd.date_range('2023-01-01', periods=24*60, freq='H')  # 60 days hourly
        days = hours
        trend = 0.01 * np.arange(len(hours))  # Gradual increase
        noise = np.random.normal(0, 2, len(hours))
        # Add sudden spikes (anomalies)
        spikes = np.zeros(len(hours))
        spike_indices = np.random.choice(len(hours), size=10, replace=False)
        spikes[spike_indices] = np.random.uniform(10, 20, 10)
        values = 50 + trend + noise + spikes
        label = 'Machine Vibration (mm/s)'
        
    else:  # finance
        # Revenue - quarterly seasonality with trend
        months = pd.date_range('2020-01-01', periods=48, freq='MS')  # 4 years monthly
        days = months
        trend = np.linspace(1000, 1500, 48)
        quarterly_season = 200 * np.sin(2 * np.pi * np.arange(48) / 12)
        noise = np.random.normal(0, 50, 48)
        values = trend + quarterly_season + noise
        label = 'Monthly Revenue ($1000s)'
    
    return pd.DataFrame({'date': days, 'value': values}), label

# SELECT YOUR SCENARIO HERE
scenario = 'retail'  # Change to: 'retail', 'energy', 'maintenance', or 'finance'

df_scenario, value_label = create_scenario_data(scenario)

# Visualize scenario data
plt.figure(figsize=(15, 6))
plt.plot(df_scenario['date'], df_scenario['value'], linewidth=1)
plt.title(f'Scenario: {scenario.upper()} - Historical Data', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel(value_label)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📋 Your scenario: {scenario.upper()}")
print(f"Dataset size: {len(df_scenario)} observations")
print(f"Date range: {df_scenario['date'].min()} to {df_scenario['date'].max()}")

### 📝 Group Task (15 minutes)

Complete the following steps with your group:

1. **Analyze** the historical data
2. **Determine** ARIMA parameters (p, d, q)
3. **Fit** the model
4. **Make** forecasts with confidence intervals
5. **Evaluate** performance (MAE, RMSE)

Use the code cell below for your analysis:

In [ ]:
# GROUP WORK SPACE

# Step 1: Check stationarity
print("=" * 50)
print("STEP 1: STATIONARITY CHECK")
print("=" * 50)
is_stationary = adf_test(df_scenario['value'], 'Scenario Data')

# Step 2: ACF/PACF plots
print("\n" + "=" * 50)
print("STEP 2: ACF/PACF ANALYSIS")
print("=" * 50)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
series_for_plots = df_scenario['value'].diff().dropna() if not is_stationary else df_scenario['value']
plot_acf(series_for_plots, lags=40, ax=axes[0])
axes[0].set_title('ACF')
plot_pacf(series_for_plots, lags=40, ax=axes[1])
axes[1].set_title('PACF')
plt.tight_layout()
plt.show()

# Step 3: Choose your parameters
print("\n" + "=" * 50)
print("STEP 3: CHOOSE PARAMETERS")
print("=" * 50)

# YOUR PARAMETERS HERE (discuss with your group)
p = 1  # AR order
d = 1  # Differencing
q = 1  # MA order

print(f"\nChosen parameters: ARIMA({p}, {d}, {q})")

# Step 4: Fit model
print("\n" + "=" * 50)
print("STEP 4: FIT MODEL")
print("=" * 50)

model = ARIMA(df_scenario['value'], order=(p, d, q))
fitted_model = model.fit()

# Step 5: Make forecast
print("\n" + "=" * 50)
print("STEP 5: FORECAST")
print("=" * 50)

forecast_steps = 30  # Adjust based on your scenario
forecast = fitted_model.forecast(steps=forecast_steps)
forecast_obj = fitted_model.get_forecast(steps=forecast_steps)
forecast_ci = forecast_obj.conf_int()

# Determine forecast index frequency based on original data
freq = pd.infer_freq(df_scenario['date'])
if freq is None:
    freq = 'D'  # Default to daily

forecast_index = pd.date_range(start=df_scenario['date'].iloc[-1] + pd.Timedelta(days=1), 
                               periods=forecast_steps, freq=freq)

# Visualize
plt.figure(figsize=(15, 6))
plt.plot(df_scenario['date'], df_scenario['value'], label='Historical', linewidth=2)
plt.plot(forecast_index, forecast, label='Forecast', linewidth=2, color='red', linestyle='--')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], 
                 alpha=0.3, color='red', label='95% CI')
plt.axvline(x=df_scenario['date'].iloc[-1], color='black', linestyle=':', linewidth=2)
plt.title(f'Forecast: {scenario.upper()}', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel(value_label)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Step 6: Evaluate
fitted_values = fitted_model.fittedvalues
mae = mean_absolute_error(df_scenario['value'][d:], fitted_values[d:])
rmse = np.sqrt(mean_squared_error(df_scenario['value'][d:], fitted_values[d:]))

print(f"\n📊 MODEL PERFORMANCE:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"AIC: {fitted_model.aic:.2f}")
print(f"BIC: {fitted_model.bic:.2f}")

### 📊 Prepare Your Findings for Gallery Walk

**Key findings to share:**

1. **Scenario:** ___________
2. **ARIMA Parameters:** (p, d, q) = ___________
3. **Reasoning:** Why did you choose these parameters? ___________
4. **Forecast:** What is your prediction? ___________
5. **Confidence:** How certain are you? (Look at CI width) ___________
6. **Challenges:** What was difficult? ___________
7. **MAE/RMSE:** ___________

### 🚶 Gallery Walk (10 minutes)

- Display your results
- Walk around and view other groups' work
- Place feedback/questions on post-its
- Discuss: Why do forecasts differ between groups?

---
## Part 3: Time Series Motifs (Pattern Mining) (30 min)

### Motivation

**"Forecasting is important, but sometimes we want to recognize recurring patterns"**

**Applications:**
- ECG: Recognize heartbeat patterns
- Website traffic: Detect user behavior patterns
- Manufacturing: Predict quality issues from recurring patterns

### What are Motifs?

**Motifs** = Subsequences that occur frequently in a time series

**Applications:**
- Anomaly detection (find patterns that DON'T match)
- Clustering similar time windows
- Classification based on pattern presence

**Techniques:**
- Matrix Profile
- Symbolic representation (SAX)
- Distance-based methods

In [ ]:
# Generate machine sensor data with repeating patterns
np.random.seed(42)

def create_pattern(length=50):
    """Create a distinct pattern"""
    return 10 * np.sin(2 * np.pi * np.arange(length) / 10) + np.random.normal(0, 1, length)

# Create synthetic sensor data
n_points = 1000
sensor_data = np.random.normal(50, 3, n_points)

# Pattern 1: Normal operation (appears multiple times)
pattern_normal = create_pattern(50)
positions_normal = [100, 300, 500, 700]
for pos in positions_normal:
    sensor_data[pos:pos+50] = 50 + pattern_normal

# Pattern 2: Start/stop phase
pattern_startstop = 5 * np.exp(-np.arange(50)/10) * np.sin(np.arange(50)/2)
positions_startstop = [200, 600]
for pos in positions_startstop:
    sensor_data[pos:pos+50] = 50 + pattern_startstop

# Pattern 3: Potential issue (spike pattern)
pattern_issue = np.zeros(50)
pattern_issue[20:30] = 15
positions_issue = [400, 800]
for pos in positions_issue:
    sensor_data[pos:pos+50] = 50 + pattern_issue + np.random.normal(0, 2, 50)

# Create DataFrame
time = pd.date_range('2023-01-01', periods=n_points, freq='T')
df_sensor = pd.DataFrame({
    'time': time,
    'value': sensor_data
})

# Visualize
plt.figure(figsize=(15, 6))
plt.plot(df_sensor['time'], df_sensor['value'], linewidth=0.5, alpha=0.8)

# Highlight pattern locations
for pos in positions_normal:
    plt.axvspan(time[pos], time[pos+49], alpha=0.2, color='green', label='Normal' if pos == positions_normal[0] else '')
for pos in positions_startstop:
    plt.axvspan(time[pos], time[pos+49], alpha=0.2, color='blue', label='Start/Stop' if pos == positions_startstop[0] else '')
for pos in positions_issue:
    plt.axvspan(time[pos], time[pos+49], alpha=0.2, color='red', label='Issue' if pos == positions_issue[0] else '')

plt.title('Machine Sensor Readings (with hidden patterns)', fontsize=16, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Sensor Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🎯 Goal: Find the top-3 most common patterns (motifs) in this data!")

### Motif Detection with Matrix Profile

We'll use the **stumpy** library for efficient motif detection.

In [ ]:
if STUMPY_AVAILABLE:
    # Compute matrix profile
    window_size = 50  # Size of patterns to look for
    matrix_profile = stumpy.stump(df_sensor['value'], m=window_size)
    
    # Find top-3 motifs
    motifs = stumpy.motifs(df_sensor['value'], matrix_profile[:, 0], max_motifs=3, max_matches=10)
    
    # Visualize motifs
    fig, axes = plt.subplots(4, 1, figsize=(15, 12))
    
    # Original data
    axes[0].plot(df_sensor['value'], linewidth=0.5, alpha=0.7)
    axes[0].set_title('Original Sensor Data', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Value')
    axes[0].grid(True, alpha=0.3)
    
    # Plot each motif
    colors = ['red', 'blue', 'green']
    for i in range(min(3, len(motifs))):
        motif_indices = motifs[i]
        
        # Highlight motif locations in original data
        for idx in motif_indices:
            axes[0].axvspan(idx, idx + window_size, alpha=0.2, color=colors[i])
        
        # Plot motif instances
        for j, idx in enumerate(motif_indices):
            axes[i+1].plot(df_sensor['value'].iloc[idx:idx+window_size].values, 
                          alpha=0.7, label=f'Instance {j+1} (pos {idx})')
        
        axes[i+1].set_title(f'Motif {i+1}: Found {len(motif_indices)} instances', 
                           fontsize=12, fontweight='bold', color=colors[i])
        axes[i+1].set_ylabel('Value')
        axes[i+1].legend(loc='upper right')
        axes[i+1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🔍 Motif Detection Results:")
    for i, motif_indices in enumerate(motifs[:3]):
        print(f"\nMotif {i+1}:")
        print(f"  - Found {len(motif_indices)} instances")
        print(f"  - Locations: {motif_indices}")
else:
    print("⚠️  Stumpy not available. Using alternative simple pattern detection...")
    print("\nInstall stumpy with: pip install stumpy")
    print("\nAlternatively, we'll use a simple sliding window correlation approach:")
    
    # Simple pattern detection using correlation
    window_size = 50
    correlations = []
    
    # Take first window as template
    template = df_sensor['value'].iloc[100:150].values
    
    # Find similar patterns
    similar_positions = []
    threshold = 0.8
    
    for i in range(len(df_sensor) - window_size):
        window = df_sensor['value'].iloc[i:i+window_size].values
        corr = np.corrcoef(template, window)[0, 1]
        if corr > threshold and i > 50:  # Avoid overlapping with template
            similar_positions.append(i)
    
    print(f"\n🔍 Found {len(similar_positions)} similar patterns to template at position 100")
    print(f"Locations: {similar_positions[:5]}...")  # Show first 5

### 📝 Exercise: Link Motifs to Events

Look at the motifs discovered above and answer:

**Motif 1:**
- Pattern shape: ___________
- Likely represents: ☐ Normal operation ☐ Start/Stop ☐ Issue
- Reasoning: ___________

**Motif 2:**
- Pattern shape: ___________
- Likely represents: ☐ Normal operation ☐ Start/Stop ☐ Issue
- Reasoning: ___________

**Motif 3:**
- Pattern shape: ___________
- Likely represents: ☐ Normal operation ☐ Start/Stop ☐ Issue
- Reasoning: ___________

### 🤔 Reflection: Predictive Maintenance Application

**How could you use motif detection for predictive maintenance?**

Ideas:
1. ___________________________________________
2. ___________________________________________
3. ___________________________________________

**Discussion prompt:** What pattern would indicate an upcoming failure?

### 👥 Peer Discussion (5 minutes)

Compare with your neighbor:

1. **Which motifs did you each find?**
2. **Do you agree on the interpretation?**
3. **What's the most interesting insight?**

Be ready to share with the class!

---
## Integration & Reflection (5 min)

### Group Discussion: When to use ARIMA vs Motif Analysis?

| Technique | Best Used For | Example |
|-----------|--------------|----------|
| **ARIMA** | Forecasting future values | Predicting next month's sales |
| **Motif Analysis** | Pattern recognition, anomaly detection | Finding normal vs abnormal machine behavior |

### 📝 Final Reflection

**Post your answers (individually):**

1. **What is the difference between forecasting and pattern recognition?**

   ___________________________________________

2. **Name ONE practical application from today that you find interesting:**

   ___________________________________________

3. **What do you want to learn more about regarding time series?**

   ___________________________________________

---
## 🎯 Key Takeaways

### ARIMA Forecasting:
1. **AR (p)**: Past values influence future (autoregressive)
2. **I (d)**: Differencing for stationarity
3. **MA (q)**: Past errors influence future (moving average)
4. **Process**: Check stationarity → ACF/PACF → Fit → Forecast → Evaluate
5. **Always include confidence intervals!**

### Pattern Mining:
1. **Motifs**: Frequently occurring subsequences
2. **Applications**: Anomaly detection, clustering, classification
3. **Matrix Profile**: Efficient method for motif discovery
4. **Practical use**: Recognize normal patterns to detect abnormal ones

---

## 🚀 Next Steps

**Further learning:**
- SARIMA for seasonal patterns
- Prophet for trend+seasonality
- Deep learning (LSTM) for complex time series

**Additional resources:**
- [Statsmodels documentation](https://www.statsmodels.org/)
- [Stumpy Matrix Profile](https://stumpy.readthedocs.io/)
- Time series papers and tutorials

---

## Thank you! 👋

**Great work today on forecasting and pattern mining!**